
# Mô hình chuỗi và Long-Short Term Memory Networks

Trong LSTM,  ứng với mỗi phần tử trong chuỗi là một *hidden state* $h_t$, chứa các thông tin về các điểm bất kỳ trước đó trong chuỗi. 

Input trong Pytorch's LSTM là tensors 3 chiều. Chiều đầu tiên là chính bản thân cái chuỗi, chiều thứ hai đánh số (index) các trường hợp trong mini-batch, và chiều thứ 3 đánh số các phần tử của input.

so lets just ignore that
and assume we will always have just 1 dimension on the second axis. If
we want to run the sequence model over the sentence "The cow jumped",
our input should look like

\begin{align}\begin{bmatrix}
   \overbrace{q_\text{The}}^\text{row vector} \\
   q_\text{cow} \\
   q_\text{jumped}
   \end{bmatrix}\end{align}

Except remember there is an additional 2nd dimension with size 1.

In addition, you could go through the sequence one at a time, in which
case the 1st axis will have size 1 also.

Let's see a quick example.



In [1]:
# tác giả: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
# input và output đều có 3 chiều: 
lstm = nn.LSTM(3, 3)  

# tạo chuỗi có độ dài là 5:
inputs = [torch.randn(1, 3) for _ in range(5)]  

# khởi tạo hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # lần lượt đi qua từng phần tử của chuỗi 
    # sau  mỗi bước, hidden được lưu trong hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

Ta cũng có thể làm cho cả chuỗi cùng lúc. Giá trị đầu tiên được LSTM trả về là tất cả các  hidden states trong chuỗi. Cái thứ hài là hidden state gần nht (so sánh cái cuối cùng của out và hidden ở dưới, ta thấy chúng giống nhau).

Đó là vì out cho ta tiếp cận tất cả các hidden states trong chuỗi. Hidden cho ta tiếp tục chuỗi và backpropagate, bằng cách pass nó như 1 argument vào lstm ở một thời điểm sau đó

In [3]:
# Thêm vào chiều thứ hai:
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
# clean out hidden state
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<CatBackward>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<ViewBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<ViewBackward>))


## Ví dụ: LSTM cho Part-of-Speech Tagging


In this section, we will use an LSTM to get part of speech tags. We will
not use Viterbi or Forward-Backward or anything like that, but as a
(challenging) exercise to the reader, think about how Viterbi could be
used after you have seen what is going on.

The model is as follows: let our input sentence be
$w_1, \dots, w_M$, where $w_i \in V$, our vocab. Also, let
$T$ be our tag set, and $y_i$ the tag of word $w_i$.
Denote our prediction of the tag of word $w_i$ by
$\hat{y}_i$.

This is a structure prediction, model, where our output is a sequence
$\hat{y}_1, \dots, \hat{y}_M$, where $\hat{y}_i \in T$.

To do the prediction, pass an LSTM over the sentence. Denote the hidden
state at timestep $i$ as $h_i$. Also, assign each tag a
unique index (like how we had word\_to\_ix in the word embeddings
section). Then our prediction rule for $\hat{y}_i$ is

\begin{align}\hat{y}_i = \text{argmax}_j \  (\log \text{Softmax}(Ah_i + b))_j\end{align}

That is, take the log softmax of the affine map of the hidden state,
and the predicted tag is the tag that has the maximum value in this
vector. Note this implies immediately that the dimensionality of the
target space of $A$ is $|T|$.



In [4]:
training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
training_data

[(['The', 'dog', 'ate', 'the', 'apple'], ['DET', 'NN', 'V', 'DET', 'NN']),
 (['Everybody', 'read', 'that', 'book'], ['NN', 'V', 'DET', 'NN'])]

In [5]:
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

{'Everybody': 5, 'ate': 2, 'apple': 4, 'that': 7, 'read': 6, 'dog': 1, 'book': 8, 'the': 3, 'The': 0}


In [6]:
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the 
# weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

In [7]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

Create the model:



In [8]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, 
                 vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # LSTM lấy word embeddings làm input và cho ra output 
        # là hidden states với số chiều là hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # layer tuyến tính maps từ không gian các 
        # hidden state sang tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Trước khi chạy, ta không có hidden state nên phải khởi tạo
        # các chiều là (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Train mô hình:



In [9]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM,
                   len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# phần tử ở vị trí i,j của output là score cho tag j cho word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

tensor([[-1.1389, -1.2024, -0.9693],
        [-1.1065, -1.2200, -0.9834],
        [-1.1286, -1.2093, -0.9726],
        [-1.1190, -1.1960, -0.9916],
        [-1.0137, -1.2642, -1.0366]])


In [10]:
for epoch in range(100):
    for sentence, tags in training_data:
        # clear các gradient của các lần chạy trước
        model.zero_grad()

        # clear các hidden state của LSTM,
        # tách nó khỏi lịch sử của vòng lặp trước
        model.hidden = model.init_hidden()

        # biến input thành các tensor chứa các word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # chạy forward pass.
        tag_scores = model(sentence_in)

        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

In [11]:

with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    # kết quả của dự đoán là tag có score cao nhất
    # The sentence is "the dog ate the apple". i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

tensor([[-0.6973, -1.2696, -1.5089],
        [-2.7420, -0.2178, -2.0302],
        [-1.4045, -1.7772, -0.5354],
        [-0.2387, -2.2666, -2.2193],
        [-2.7822, -0.1865, -2.2232]])
